We start with seven fields:
Date - specifies trading date
Open - opening price
High - maximum price during the day
Low - minimum price during the day
Close - close price adjusted for splits
Adj Close - adjusted close price adjusted for both dividends and splits.
Volume - the number of shares that changed hands during a given day

https://www.kaggle.com/datasets/jacksoncrow/stock-market-dataset?resource=download-directory&select=stocks 

We want to predict seven fields?

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

from pathlib import Path
import pandas
import nasdaqdatalink as datalink
NASDAQ_DATA_LINK_API_KEY = "XBH69AuacmmTQ5-HbcSi"

In [ ]:
# load dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# Training dataset

# Custom Dataset class for our custom data
class StockDataset(Dataset):
    def __init__(self, csvFolder):
        self._stockNames = []
        # Get each name and add to array
        for file in Path(csvFolder).iterdir():
            if file.is_file():
                self._stockNames.append(file.name)

    def __len__(self):
        return len(self._stockNames)

    def __getitem__(self, index):
        # Use index to read corresponding file
        csvPath = Path(csvFolder) / self._stockNames[index]
        # Converts .csv data to tensor
        stockData = torch.from_numpy(pandas.read_csv(csvPath).values)
        return stockData

train_dataset_path = "C:/Users/simsd/projects/StockPrices/stocks" # Put training folder path here
train_dataset = StockDataset(train_dataset_path)
#print(train_dataset._stockNames) # Accessing "private" class member

# Testing dataset
test_dataset = datalink

['A.csv', 'AA.csv', 'AACG.csv', 'AAL.csv', 'AAMC.csv', 'AAME.csv', 'AAN.csv', 'AAOI.csv', 'AAON.csv', 'AAP.csv', 'AAPL.csv', 'AAT.csv', 'AAU.csv', 'AAWW.csv', 'AAXN.csv', 'AB.csv', 'ABB.csv', 'ABBV.csv', 'ABC.csv', 'ABCB.csv', 'ABEO.csv', 'ABEV.csv', 'ABG.csv', 'ABIO.csv', 'ABM.csv', 'ABMD.csv', 'ABR.csv', 'ABT.csv', 'ABTX.csv', 'ABUS.csv', 'AC.csv', 'ACA.csv', 'ACAD.csv', 'ACAM.csv', 'ACAMU.csv', 'ACB.csv', 'ACBI.csv', 'ACC.csv', 'ACCO.csv', 'ACEL.csv', 'ACER.csv', 'ACGL.csv', 'ACGLO.csv', 'ACGLP.csv', 'ACH.csv', 'ACHC.csv', 'ACHV.csv', 'ACIA.csv', 'ACIU.csv', 'ACIW.csv', 'ACLS.csv', 'ACM.csv', 'ACMR.csv', 'ACN.csv', 'ACNB.csv', 'ACOR.csv', 'ACP.csv', 'ACRE.csv', 'ACRS.csv', 'ACRX.csv', 'ACST.csv', 'ACTG.csv', 'ACTT.csv', 'ACTTU.csv', 'ACU.csv', 'ACV.csv', 'ACY.csv', 'ADAP.csv', 'ADBE.csv', 'ADC.csv', 'ADES.csv', 'ADI.csv', 'ADIL.csv', 'ADM.csv', 'ADMA.csv', 'ADMP.csv', 'ADMS.csv', 'ADNT.csv', 'ADP.csv', 'ADPT.csv', 'ADRO.csv', 'ADS.csv', 'ADSK.csv', 'ADSW.csv', 'ADT.csv', 'ADTN.csv',

In [25]:
# Define the neural network class
class NeuralNet(nn.Module):
    def __init__(self):
        #super(NeuralNet, self).__init__()
        #1st  layer: map 7 inputs to (32) hidden neurons
        self.fc1 = nn.Linear(7, 32)
        #2nd layer: map (32) hidden neurons to (32) hidden neurons
        self.fc2 = nn.Linear(32, 32)
        #3rd layer: map (32) hidden neurons to (7?) outputs representing future stock prices
        self.fc3 = nn.Linear(32, 7)

# Use “forward” to define the computing flow. ReLu activation is applied. 
    def forward(self, x):
        #x = x.view(-1, 28 * 28)  
        # First layer computation
        x = F.relu(self.fc1(x))
        # Second layer computation
        x = F.relu(self.fc2(x))
        # Third layer computation
        x = self.fc3(x)
        return x